# Polaroid REST /exec Demo (QuestDB-like)

This notebook demonstrates the new HTTP API exposed by `polaroid-grpc`:

- `GET /ping` health check
- `GET /exec` in **QuestDB proxy mode** (`query=`)
- `GET /exec` in **handle mode** (`handle=`) (requires a handle created via gRPC)

## Prerequisites

1) Start the server in a terminal:

```bash
cd polaroid
cargo run -p polaroid-grpc
```

2) Optional: configure QuestDB proxy mode by setting one of:
- `POLAROID_QUESTDB_HTTP_URL`
- `QUESTDB_HTTP_URL`

Example:
```bash
export POLAROID_QUESTDB_HTTP_URL="http://localhost:9000"
```

In [ ]:
import os
import json

try:
    import requests
except Exception as e:
    raise RuntimeError("Please install requests: pip install requests") from e

POLAROID_HTTP_URL = os.environ.get('POLAROID_HTTP_URL', 'http://localhost:9000')
print('POLAROID_HTTP_URL =', POLAROID_HTTP_URL)


In [ ]:
# Health check
r = requests.get(f"{POLAROID_HTTP_URL}/ping")
print('status:', r.status_code)
print('body:', r.text)


## QuestDB proxy mode (`query=`)

If QuestDB is configured via `POLAROID_QUESTDB_HTTP_URL`/`QUESTDB_HTTP_URL`, Polaroid will proxy:
`/exec?query=...&fmt=json` → `${QUESTDB}/exec?query=...&fmt=json`.

If it is not configured, Polaroid returns **HTTP 412** with a configuration hint.

In [ ]:
# QuestDB proxy mode
r = requests.get(f"{POLAROID_HTTP_URL}/exec", params={"query": "select 42", "fmt": "json"})
print('status:', r.status_code)
try:
    print(json.dumps(r.json(), indent=2)[:4000])
except Exception:
    print(r.text[:4000])


## Handle mode (`handle=`)

Handle mode returns a QuestDB-like JSON representation of a DataFrame that is already in the server’s handle store.

To use this mode you need a handle (created by a gRPC call such as `LoadParquet`, `FetchRestApi`, etc. depending on what you’ve enabled).

Once you have a handle, call:
`/exec?handle=<HANDLE>&fmt=json&limit=1000`.

In [ ]:
# Replace with a real handle you obtained from gRPC
HANDLE = os.environ.get('POLAROID_HANDLE')
if not HANDLE:
    print('Set POLAROID_HANDLE env var to test handle mode.')
else:
    r = requests.get(f"{POLAROID_HTTP_URL}/exec", params={"handle": HANDLE, "fmt": "json", "limit": 1})
    print('status:', r.status_code)
    print(r.text[:4000])


## Next steps

- Add local SQL execution (DataFusion) so `/exec?query=...` works without QuestDB.
- Add Ballista submission so `/exec?query=...` can run distributed.
- Add output formats: Arrow IPC + CSV.
- Add auth/multi-tenant handle namespaces.